# TAREA 4
Blanca Cecilia Ubaque - 201725992

María Carolina Trespalacios – 200313688

# Exercise 04

# Fraud Detection

## Introduction

- Fraud Detection Dataset from Microsoft Azure: [data](http://gallery.cortanaintelligence.com/Experiment/8e9fe4e03b8b4c65b9ca947c72b8e463)

Fraud detection is one of the earliest industrial applications of data mining and machine learning. Fraud detection is typically handled as a binary classification problem, but the class population is unbalanced because instances of fraud are usually very rare compared to the overall volume of transactions. Moreover, when fraudulent transactions are discovered, the business typically takes measures to block the accounts from transacting to prevent further losses. 

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [3]:
import zipfile
with zipfile.ZipFile('../datasets/fraud_detection.csv.zip', 'r') as z:
    f = z.open('15_fraud_detection.csv')
    data = pd.io.parsers.read_table(f, index_col=0, sep=',')

In [4]:
data.head()

,accountAge,digitalItemCount,sumPurchaseCount1Day,sumPurchaseAmount1Day,sumPurchaseAmount30Day,paymentBillingPostalCode - LogOddsForClass_0,accountPostalCode - LogOddsForClass_0,paymentBillingState - LogOddsForClass_0,accountState - LogOddsForClass_0,paymentInstrumentAgeInAccount,ipState - LogOddsForClass_0,transactionAmount,transactionAmountUSD,ipPostalCode - LogOddsForClass_0,localHour - LogOddsForClass_0,Label
0,2000,0,0,0.00,720.25,5.064533,0.421214,1.312186,0.566395,3279.574306,1.218157,599.00,626.164650,1.259543,4.745402,0
1,62,1,1,1185.44,2530.37,0.538996,0.481838,4.401370,4.500157,61.970139,4.035601,1185.44,1185.440000,3.981118,4.921349,0
2,2000,0,0,0.00,0.00,5.064533,5.096396,3.056357,3.155226,0.000000,3.314186,32.09,32.090000,5.008490,4.742303,0
3,1,1,0,0.00,0.00,5.064533,5.096396,3.331154,3.331239,0.000000,3.529398,133.28,132.729554,1.324925,4.745402,0
4,1,1,0,0.00,132.73,5.412885,0.342945,5.563677,4.086965,0.001389,3.529398,543.66,543.660000,2.693451,4.876771,0


In [5]:
data.shape, data.Label.sum(), data.Label.mean()

((138721, 16), 797, 0.0057453449730033666)

In [6]:
X = data.drop(['Label'], axis=1)
y = data['Label']

In [7]:
# Check for missing values
data.isnull().sum()

accountAge                                      0
digitalItemCount                                0
sumPurchaseCount1Day                            0
sumPurchaseAmount1Day                           0
sumPurchaseAmount30Day                          0
paymentBillingPostalCode - LogOddsForClass_0    0
accountPostalCode - LogOddsForClass_0           0
paymentBillingState - LogOddsForClass_0         0
accountState - LogOddsForClass_0                0
paymentInstrumentAgeInAccount                   0
ipState - LogOddsForClass_0                     0
transactionAmount                               0
transactionAmountUSD                            0
ipPostalCode - LogOddsForClass_0                0
localHour - LogOddsForClass_0                   0
Label                                           0
dtype: int64

# Exercice 04.1

Estimate a Logistic Regression

Evaluate using the following metrics:
* Accuracy
* F1-Score
* F_Beta-Score (Beta=10)

Comment about the results

In [8]:
# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# train a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train, y_train)

# make predictions for testing set
y_pred_class = logreg.predict(X_test)

C:\Users\tresp\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_class)

array([[34472,     0],
       [  209,     0]], dtype=int64)

In [10]:
# calculate testing accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.9939736455119518


In [11]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision_score ', precision_score(y_test, y_pred_class))
print('recall_score    ', recall_score(y_test, y_pred_class))

precision_score  0.0
recall_score     0.0


C:\Users\tresp\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
print('f1_score    ', f1_score(y_test, y_pred_class))

f1_score     0.0


C:\Users\tresp\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
# Dado el desbalance de la muestra, el modelo solo está prediciendo en la clase mayoritaria, por lo que se requiere aplicar un método de rebalanceo.

# Exercice 04.2

Under-sample the negative class using random-under-sampling

Which is parameter for target_percentage did you choose?
How the results change?

**Only apply under-sampling to the training set, evaluate using the whole test set**

In [14]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [15]:
y_train.mean()
# Se observa que los casos positivos de fraude son muy pocos con respecto al total de la muestra

0.0056516724336793545

In [16]:
def UnderSampling(X_train, y_train, target_percentage=0.5, seed=None):
    # Assuming minority class is the positive
    n_samples = y_train.shape[0]
    n_samples_0 = (y_train == 0).sum()
    n_samples_1 = (y_train == 1).sum()

    n_samples_0_new =  n_samples_1 / target_percentage - n_samples_1
    n_samples_0_new_per = n_samples_0_new / n_samples_0

    filter_ = y_train == 0

    np.random.seed(seed)
    rand_1 = np.random.binomial(n=1, p=n_samples_0_new_per, size=n_samples)
    
    filter_ = filter_ & rand_1
    filter_ = filter_ | (y == 1)
    filter_ = filter_.astype(bool)
    
    return X_train[filter_], y_train[filter_]

In [17]:
for target_percentage in [0.1, 0.2, 0.3, 0.4, 0.5]:
    X_u, y_u = UnderSampling(X_train, y_train, target_percentage, 1)
    print('Target percentage', target_percentage)
    print('y.shape = ',y_u.shape[0], 'y.mean() = ', y_u.mean())

C:\Users\tresp\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Target percentage 0.1
y.shape =  5867 y.mean() =  0.10022157831941367
Target percentage 0.2
y.shape =  2939 y.mean() =  0.20006805035726438
Target percentage 0.3
y.shape =  1920 y.mean() =  0.30625
Target percentage 0.4
y.shape =  1430 y.mean() =  0.4111888111888112
Target percentage 0.5
y.shape =  1151 y.mean() =  0.5108601216333623


In [18]:
# Se escoge un porcentaje de positivos alrededor de 0.5
X_u, y_u = UnderSampling(X_train, y_train, 0.5, 1)

C:\Users\tresp\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [19]:
# Se tabulan las frecuencias de cada clase en la nueva muestra con undersampling
pd.crosstab(index=y_u, columns="count")

col_0,count
Label,
0,563
1,588


# Exercice 04.3

Now using random-over-sampling

In [20]:
!pip install imblearn

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [21]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_sample(X_train, y_train)

In [22]:
# Se tabulan las frecuencias de cada clase en la nueva muestra con undersampling
pd.crosstab(index=y_resampled, columns="count")

col_0,count
row_0,
0,103452
1,103452


# Exercice 04.4*
Evaluate the results using SMOTE

Which parameters did you choose?

In [23]:
from imblearn.over_sampling import SMOTE

# Apply regular SMOTE
sm = SMOTE('regular')

X_res, y_res = sm.fit_sample(X_train, y_train)

ValueError: When 'ratio' is a string, it needs to be one of {'minority': <function _ratio_minority at 0x00000187BA47AD08>, 'majority': <function _ratio_majority at 0x00000187BA47ABF8>, 'not minority': <function _ratio_not_minority at 0x00000187BA47AC80>, 'all': <function _ratio_all at 0x00000187BA47AB70>, 'auto': <function _ratio_auto at 0x00000187BA47AD90>}. Got 'regular' instead.

# Exercice 04.5

Estimate a Logistic Regression, GaussianNB, K-nearest neighbors and a Decision Tree **Classifiers**

Evaluate using the following metrics:
* Accuracy
* F1-Score
* F_Beta-Score (Beta=10)

Comment about the results

Combine the classifiers and comment

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [28]:
# Se crea un diccionario para facilitar la estimación de los modelos requeridos
mod = {'lr': LogisticRegression(),
          'dt': DecisionTreeClassifier(),
          'nb': GaussianNB(),
          'nn': KNeighborsClassifier()}

In [29]:
# Train all the models
for model in mod.keys():
    mod[model].fit(X_train, y_train)

In [30]:
# predict test for each model
y_hat = pd.DataFrame(index=y_test.index, columns=models.keys())
for model in mod.keys():
    y_hat[model] = mod[model].predict(X_test)

In [31]:
# Evaluate each model accuracy
for model in mod.keys():
    print(model,float((y_hat[model] == y_test).mean()))

lr 0.9939736455119518
dt 0.9890141576079121
nb 0.9913785646319311
nn 0.9937429716559499


El modelo con mejor accuracy es el clasificador por k neighbors

In [32]:
# Evaluate each model F1 Score
from sklearn.metrics import f1_score
for model in mod.keys():
    print(model, f1_score(y_test, y_hat[model]))

lr 0.0
dt 0.13995485327313772
nb 0.0
nn 0.1422924901185771


C:\Users\tresp\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


El modelo con el cual se maximiza el f1_score es el clasificador por k neighbors

# Exercice 04.6

Using the under-sampled dataset

Evaluate a RandomForestClassifier and compare the results

change n_estimators=100, what happened

In [33]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [34]:
# Se estima el modelo con la muestra con undersampling
rf.fit(X_u, y_u)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [35]:
# Se calcula el MSE
scores = cross_val_score(rf, X_u, y_u, cv=10, scoring='neg_mean_squared_error')
np.mean(np.sqrt(-scores))

0.4464464351450384

In [38]:
# Se creó un nuevo modelo con 100 estimadores 
rf2 = RandomForestRegressor(n_estimators=100)
rf2
rf2.fit(X_u, y_u)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [39]:
#Nuevamente se calculó el MSE
scores2 = cross_val_score(rf2, X_u, y_u, cv=10, scoring='neg_mean_squared_error')
np.mean(np.sqrt(-scores2))

0.4370568136698939

Se observa que el modelo mejoró con 100 estimadores.